In this notebook, we demostrated how to run an inference and how to use it.

In [ ]:
import ray
import os

In [ ]:
import sys; sys.path.append("..")
import warnings; warnings.filterwarnings("ignore")
from dotenv import load_dotenv; load_dotenv()
%load_ext autoreload
%autoreload 2

### Start ray cluster

In [ ]:
# Credentials
ray.init(
    runtime_env={
        "env_vars": {
        },
        "working_dir": "/Users/lipeng/workspaces/git-devops/llm-ray"
    },
    num_cpus=8,
)

In [ ]:
# ray.shutdown()

In [ ]:
ray.cluster_resources()

### Start single inference with UI
`llm_experimental` will start a LLM inference, besides that, it will create simple UI based on the downstream `task` in the model defination file(yaml), you can:
- Open UI(will print at the end of logs) in browser
- Adopt `DeploymentHandle` to invoke the function supplied by `llm_experimental`. i.e. `query`

In [ ]:
from llmserve.backend.server.run import llm_experimental
from ray import serve
from ray.serve.handle import DeploymentHandle, DeploymentResponse

app_suite = llm_experimental("../models/text-generation--gpt2.yaml")
serve_name = app_suite[1]["name"]
app = app_suite[0]
handle: DeploymentHandle = serve.run(app, name=serve_name, route_prefix="/" + serve_name)
print("done")

In [ ]:
response = handle.query.remote("Python is kind of")
print(ray.get(response))

Delete inference

In [ ]:
serve.delete(serve_name, _blocking = True)

### Start multiple inferences with restful exposed

Deploy multiple inference one time, and use them with restful api

In [ ]:
from llmserve.backend.server.run import llm_server

app = llm_server(["../models/text-generation--gpt2.yaml", "../models/text-generation--facebook--opt-125m.yaml"])
serve_name = "multiple"
handle: DeploymentHandle = serve.run(app, name=serve_name, route_prefix="/" + serve_name)

In [ ]:
import requests

res = requests.get(f"http://127.0.0.1:8000/multiple/models")
print(res.json())

In [ ]:
model = "gpt2"   # pick up one model from upper cells' output for testing

data = '{"prompt": "I want to"}'
res = requests.post(f"http://127.0.0.1:8000/multiple/"+ model +"/run/predict/", data=data)
print(res.json()["generated_text"])

In [ ]:
model = "facebook/opt-125m".replace("/", "--")   # pick up one model from upper cells' output for testing

data = '{"prompt": "I want to"}'
res = requests.post(f"http://127.0.0.1:8000/multiple/"+ model +"/run/predict/", data=data)
print(res.json()["generated_text"])

Delete inference

In [ ]:
serve.delete(serve_name, _blocking = True)